### Содержание
```
1 Прямой проход и обратное распространение
2 Разбиение набора данных
3 Нормализация данных
4 Валидация и переобучение
5 Схождение
6 Контрольные точки и ранняя остановка
7 Гиперпараметры
8 Инвариантность
9 Сырые наборы данных
10 Сохранение/Восстановление модели
```

**Прямой проход и обратное распространение**

<img src="images/forward_pass.png" alt="forward" height=60% width=60%>

```
    Данные подаются в нейронную сеть пакетами (batch-ами). Пакет может состоять из одного или нескольких выбранных наугад примеров из тренировочных данных.
    Тренировочные данные подаются в модель несколько раз. Всякий раз, когда мы подаем все тренировочные данные целиком, это
называется эпохой. Каждая эпоха представляет собой отличающуюся случайную перестановку примеров в пакетах – то есть не существует двух эпох с одинаковым порядком следования примеров.
```